In [ ]:
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")' Just

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/JustHTTP/Just", from: "0.7.1")
		Just
With SwiftPM flags: []
Working in: /tmp/tmpkz26t51e
Fetching https://github.com/mxcl/Path.swift
Fetching https://github.com/JustHTTP/Just
Completed resolution in 0.99s
Cloning https://github.com/mxcl/Path.swift
Resolving https://github.com/mxcl/Path.swift at 0.16.2
Cloning https://github.com/JustHTTP/Just
Resolving https://github.com/JustHTTP/Just at 0.7.1
Compile Swift Module 'Path' (9 sources)
Compile Swift Module 'Just' (1 sources)
Compile Swift Module 'jupyterInstalledPackages' (1 sources)
Linking ./.build/x86_64-unknown-linux/debug/libjupyterInstalledPackages.so
Initializing Swift...
Loading library...
Installation complete!


## Getting the MNIST dataset

In [ ]:
// export
import Foundation
import Just
import Path

In [ ]:
//export
public func shellCommand(_ launchPath: String, _ arguments: [String]) -> String?
{
    let task = Process()
    task.executableURL = URL.init(fileURLWithPath:launchPath)
    task.arguments = arguments

    let pipe = Pipe()
    task.standardOutput = pipe
    do {try task.run()} catch {print("Unexpected error: \(error).")}

    let data = pipe.fileHandleForReading.readDataToEndOfFile()
    let output = String(data: data, encoding: String.Encoding.utf8)

    return output
}

In [ ]:
if let res = shellCommand("/bin/ls", ["-lh"]){print(res)}

total 9.7M
-rw-rw-r-- 1 ubuntu ubuntu  14K Apr  6 18:23 00_load_data.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  26K Apr  6 02:04 01_matmul.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  17K Apr  5 20:27 02_fully_connected.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  18K Apr  5 20:53 02a_why_sqrt5.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  14K Apr  6 18:20 03_minibtach_training.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  30K Mar 27 17:11 04_callbacks.ipynb
-rw-rw-r-- 1 ubuntu ubuntu  13K Apr  5 13:57 05_callbacks2.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 4.3K Apr  5 13:57 05b_early_stopping.ipynb
drwxrwxr-x 3 ubuntu ubuntu 4.0K Mar 21 20:30 FastaiNotebooks
-rw-rw-r-- 1 ubuntu ubuntu  16K Apr  5 21:00 callback_experiment.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 6.4K Mar 19 19:50 swift_nn.ipynb
-rw-rw-r-- 1 ubuntu ubuntu 9.5M Apr  6 18:20 train-images-idx3-ubyte.gz



In [ ]:
//export
public func downloadFile(_ url: String, dest: String?=nil, force: Bool=false){
    let dest_name = (dest ?? (Path.cwd/url.split(separator: "/").last!).string)
    let url_dest = URL.init(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if (force || !Path(dest_name)!.exists){
        print("Downloading \(url)...")
        if let cts = Just.get(url).content{
            do    {try cts.write(to: URL.init(fileURLWithPath:dest_name))}
            catch {print("Can't write to \(url_dest).\n\(error)")}
        } else {print("Can't reach \(url)")}
    }
}

In [ ]:
downloadFile("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")

In [ ]:
//export
import TensorFlow

In [ ]:
//export
protocol ConvertableFromByte {
    init(_ d:UInt8)
}

In [ ]:
//export
extension Float : ConvertableFromByte{}
extension Int32 : ConvertableFromByte{}

In [ ]:
func readData<T:ConvertableFromByte & TensorFlowScalar>(_ fn:String, _ skip:Int) -> Tensor<T> {
    let data = try! Data.init(contentsOf: URL.init(fileURLWithPath: fn)).dropFirst(skip)
    return Tensor(data.map(T.init))
}

In [ ]:
//export
func loadMNIST<T:ConvertableFromByte & TensorFlowScalar>(training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<T> {
    let split = training ? "train" : "t10k"
    let kind = labels ? "labels" : "images"
    let batch = training ? Int32(60000) : Int32(10000)
    let shape: TensorShape = labels ? [batch] : (flat ? [batch, 784] : [batch, 28, 28])
    let dropK = labels ? 8 : 16
    let baseUrl = "http://yann.lecun.com/exdb/mnist/"
    let fname = split + "-" + kind + "-idx\(labels ? 1 : 3)-ubyte"
    let file = path/fname
    if !file.exists {
        downloadFile("\(baseUrl)\(fname).gz", dest:(path/"\(fname).gz").string)
        _ = shellCommand("/bin/gunzip", ["-fq", (path/"\(fname).gz").string])
    }
    let data = try! Data.init(contentsOf: URL.init(fileURLWithPath: file.string)).dropFirst(dropK)
    if labels { return Tensor(data.map(T.init)) }
    else      { return Tensor(data.map(T.init)).reshaped(to: shape)}
}

public func loadMNIST(path:Path, flat:Bool = false) -> (Tensor<Float>, Tensor<Int32>, Tensor<Float>, Tensor<Int32>) {
    return (
        loadMNIST(training: true,  labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: true,  labels: true,  path: path, flat: flat),
        loadMNIST(training: false, labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: false, labels: true,  path: path, flat: flat)
    )
}

In [ ]:
//export
public let mnistPath = Path.home/".fastai"/"data"/"mnist_tst"

In [ ]:
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath)

In [ ]:
xTrain.shape

▿ TensorShape
  ▿ dimensions : 3 elements
    - 0 : 60000
    - 1 : 28
    - 2 : 28


In [ ]:
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath, flat: true)
xTrain.shape

▿ TensorShape
  ▿ dimensions : 2 elements
    - 0 : 60000
    - 1 : 784


## Timing

In [ ]:
//export 
import Dispatch
public func time(_ function: () -> ()) {
    let start = DispatchTime.now()
    function()
    let end = DispatchTime.now()
    let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
    let milliseconds = nanoseconds / 1e6
    print("\(milliseconds) ms")
}

In [ ]:
time {var valImgs: Tensor<Float> = loadMNIST(training:false, labels: false, path: mnistPath, flat:false)}

370.581335 ms


In [ ]:
//export 
public func time(repeating: Int, _ function: () -> ()) {
    function()
    var times:[Double] = []
    for _ in 1...repeating{
        let start = DispatchTime.now()
        function()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("\(times.reduce(0.0, +)/Double(times.count)) ms")
}

In [ ]:
time(repeating:10) {var valImgs: Tensor<Float> = loadMNIST(training:false, labels: false, path: mnistPath, flat:false)}

266.04751389999996 ms


## Export

In [ ]:
// export
public func notebookToScript(fname: String){
    let url_fname = URL.init(fileURLWithPath: fname)
    let last = fname.lastPathComponent
    let out_fname = (url_fname.deletingLastPathComponent().appendingPathComponent("FastaiNotebooks", isDirectory: true)
                     .appendingPathComponent("Sources", isDirectory: true)
                     .appendingPathComponent("FastaiNotebooks", isDirectory: true).appendingPathComponent(last)
                     .deletingPathExtension().appendingPathExtension("swift"))
    do{
        let data = try Data.init(contentsOf: url_fname)
        let jsonData = try! JSONSerialization.jsonObject(with: data, options: .allowFragments) as! [String: Any]
        let cells = jsonData["cells"] as! [[String:Any]]
        var module = """
/*
THIS FILE WAS AUTOGENERATED! DO NOT EDIT!
file to edit: \(fname.lastPathComponent)

*/
        
"""
        for cell in cells{
            if let source = cell["source"] as? [String]{
                if source.isEmpty {continue}
                if source[0].range(of: #"^\s*//\s*export\s*$"#, options: .regularExpression) != nil{
                    module.append("\n" + source[1...].joined() + "\n")
                }
            }
        }
        try? module.write(to: out_fname, atomically: false, encoding: .utf8)
    } catch {print("Can't read the content of \(fname)")}
}

In [ ]:
// export
public func exportNotebooks(_ path: Path){
    for entry in try! path.ls(){
        if entry.kind == Entry.Kind.file{
            if entry.path.basename().range(of: #"^\d*_.*ipynb$"#, options: .regularExpression) != nil { 
                print("Converting \(entry.path.basename())")
                notebookToScript(fname: entry.path.basename())
            }
        }
    }
}

In [ ]:
notebookToScript(fname: (Path.cwd / "00_load_data.ipynb").string)